In [2]:
import pandas as pd
import os

In [42]:
ORIGINAL_DATA_DIR = "data/original/"
ORIGINAL_GG = "data/original/golden_globes.csv"
ORIGINAL_OSCAR = "data/original/oscar.csv"
ORIGINAL_SAG = "data/original/sag.csv"
TRANS_DATA_DIR = "data/transformed/"

In [43]:
df_gg = pd.read_csv(ORIGINAL_GG)
df_oscars = pd.read_csv(ORIGINAL_OSCAR)
df_sag = pd.read_csv(ORIGINAL_SAG)

In [44]:
df_gg_2014_2024 = df_gg[(df_gg['ano'] >= 2014) & (df_gg['ano'] <= 2024)]

In [45]:
df_gg_2014_2024

,ano,nome,personagem,filme,ganhou
320,2014,Julianne Moore,Dr. Alice Howland,Still Alice,True
321,2014,Jennifer Aniston,Claire Bennett,Cake,False
322,2014,Felicity Jones,Jane Hawking,The Theory of Everything,False
323,2014,Rosamund Pike,Amy Elliott-Dunne,Gone Girl,False
324,2014,Reese Witherspoon,Cheryl Strayed,Wild,False
325,2015,Brie Larson,"Joy ""Ma"" Newsome",Room,True
326,2015,Cate Blanchett,Carol Aird,Carol,False
327,2015,Rooney Mara,Therese Belivet,Carol,False
328,2015,Saoirse Ronan,Eilis Lacey,Brooklyn,False
329,2015,Alicia Vikander,Gerda Wegener,The Danish Girl,False


In [46]:
df_gg_2014_2024['indicada_gg'] = True
df_gg_2014_2024.rename(columns={'ganhou': 'ganhou_gg'}, inplace=True)


/tmp/ipykernel_837196/1164528096.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_gg_2014_2024['indicada_gg'] = True
/tmp/ipykernel_837196/1164528096.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_gg_2014_2024.rename(columns={'ganhou': 'ganhou_gg'}, inplace=True)


In [47]:
df_gg_2014_2024.head()

,ano,nome,personagem,filme,ganhou_gg,indicada_gg
320,2014,Julianne Moore,Dr. Alice Howland,Still Alice,True,True
321,2014,Jennifer Aniston,Claire Bennett,Cake,False,True
322,2014,Felicity Jones,Jane Hawking,The Theory of Everything,False,True
323,2014,Rosamund Pike,Amy Elliott-Dunne,Gone Girl,False,True
324,2014,Reese Witherspoon,Cheryl Strayed,Wild,False,True


In [48]:
print(df_gg_2014_2024.head())

      ano               nome         personagem                     filme  \
320  2014     Julianne Moore  Dr. Alice Howland               Still Alice   
321  2014   Jennifer Aniston     Claire Bennett                      Cake   
322  2014     Felicity Jones       Jane Hawking  The Theory of Everything   
323  2014      Rosamund Pike  Amy Elliott-Dunne                 Gone Girl   
324  2014  Reese Witherspoon     Cheryl Strayed                      Wild   

     ganhou_gg  indicada_gg  
320       True         True  
321      False         True  
322      False         True  
323      False         True  
324      False         True  


Pivotar a tabela

In [49]:
df = df_gg_2014_2024

df_pivot = pd.pivot_table(
    df,
    index='nome',
    columns='ano',
    values='indicada_gg',  
    aggfunc='any',  
    fill_value=False
)

df_pivot.columns = [f'participou_{ano}' for ano in df_pivot.columns]

indicacoes = df.groupby('nome')['indicada_gg'].max()
vitorias = df.groupby('nome')['ganhou_gg'].max()

# unir os dfs
df_final = pd.concat([df_pivot, indicacoes, vitorias], axis=1)


In [50]:
df_final

,participou_2014,participou_2015,participou_2016,participou_2017,participou_2018,participou_2019,participou_2020,participou_2021,participou_2022,participou_2023,participou_2024,indicada_gg,ganhou_gg
nome,,,,,,,,,,,,,
Alicia Vikander,False,True,False,False,False,False,False,False,False,False,False,True,False
Amy Adams,False,False,True,False,False,False,False,False,False,False,False,True,False
Ana de Armas,False,False,False,False,False,False,False,False,True,False,False,True,False
Andra Day,False,False,False,False,False,False,True,False,False,False,False,True,True
Angelina Jolie,False,False,False,False,False,False,False,False,False,False,True,True,False
Annette Bening,False,False,False,False,False,False,False,False,False,True,False,True,False
Brie Larson,False,True,False,False,False,False,False,False,False,False,False,True,True
Cailee Spaeny,False,False,False,False,False,False,False,False,False,True,False,True,False
Carey Mulligan,False,False,False,False,False,False,True,False,False,True,False,True,False
